In [11]:
import cv2
import os
import numpy as np
from PIL import Image, ImageDraw, ImageFont

font_path = r"C:\Windows\Fonts\msyh.ttc"   # 微软雅黑（推荐）
# 获取当前工作目录
current_working_directory = os.getcwd()

img_dir = os.path.join(current_working_directory)
tpl_dir = os.path.join(current_working_directory)

name_czr = [0]

def getFileName (path,suffix=".jpg"):
    """
函数说明:获取指定目录下的、指定后缀的文件，读取批量jpg文件，把模板名字储存起来，对每个模板，如果能识别到，就把人名对应的状态置1，即打卡成功
    例如：.jpg
Parameters:
    path - 目录所在的路径 例如 path='D:\Python Example\Tianyancha\Data'
    suffix - 指定后缀
Returns:
    
Modify:
    2025.12.24
"""
    input_ALL = []
    f_list = os.listdir(path)
    for name in f_list:
        if os.path.splitext(name)[1] == suffix:
            input_ALL.append(name)
    #print(input_ALL)
    return input_ALL

def put_chinese_text_bottom(img_bgr, text, font_path, font_size=28,
                            margin=20, align="center", color=(0, 255, 255)):
    """
    在图片底部写中文（OpenCV BGR 输入/输出）
    align: "left" | "center" | "right"
    color: BGR
    """
    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    pil_img = Image.fromarray(img_rgb)
    draw = ImageDraw.Draw(pil_img)

    font = ImageFont.truetype(font_path, font_size)

    # 计算文字宽高
    bbox = draw.textbbox((0, 0), text, font=font)
    text_w = bbox[2] - bbox[0]
    text_h = bbox[3] - bbox[1]

    H, W = img_bgr.shape[:2]
    y = max(0, H - margin - text_h)

    if align == "left":
        x = margin
    elif align == "right":
        x = max(0, W - margin - text_w)
    else:  # center
        x = max(0, (W - text_w) // 2)

    # PIL 用的是 RGB 颜色
    draw.text((x, y), text, font=font, fill=(color[2], color[1], color[0]))

    out_bgr = cv2.cvtColor(np.array(pil_img), cv2.COLOR_RGB2BGR)
    return out_bgr

def imread_cn(img_dir, name, flags=cv2.IMREAD_COLOR):
    """
    支持 Windows 中文路径/中文文件名的读取方式
    返回: img (BGR) or None
    """
    path = os.path.join(img_dir, name)
    try:
        data = np.fromfile(path, dtype=np.uint8)   # 关键：fromfile 支持中文路径
        img = cv2.imdecode(data, flags)            # 解码成 OpenCV 图像
        return img
    except Exception as e:
        print("imread_cn error:", e, "path:", path)
        return None

# 模板检测
def Template_Match(gray_img, gray_template):
    h, w = gray_template.shape
    res = cv2.matchTemplate(gray_img, gray_template, cv2.TM_CCOEFF_NORMED)

    # 获取最大匹配位置
    min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(res)
    top_left = max_loc
    bottom_right = (top_left[0] + w, top_left[1] + h)

    # 得到分数
    , max_val, _, _ = cv2.minMaxLoc(res)
    return top_left, bottom_right, max_val



input_img = getFileName(img_dir, ".jpg")
input_template = getFileName(tpl_dir, ".jpg")


# 读取图像和模板

img = imread_cn(img_dir, input_img[0])
template = imread_cn(tpl_dir, input_template[1])

assert img is not None, "实时图片 读取失败"
assert template is not None, "模板 读取失败"


h_template, w_template = template.shape[:2]
h_img, w_img = img.shape[:2]

gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

assert (h_img >= h_template) and (w_img >= w_template), "图像尺寸必须大于模板尺寸"


tl1, br1 ,score1 = Template_Match(gray_img, gray_template)
#print(f"直接进行模板匹配的分数 {score1:.4f}")

cv2.rectangle(img, tl1, br1, (0, 255, 255), 20)  # 黄框
# cv2.putText(img, f"czr{score1:.4f}", (tl1[0], max(0, tl1[1]-10)),
#             cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 255), 2)
if score1 >= 0.9 :
    name_czr[0] = 1
if name_czr[0] == 1:
    img = put_chinese_text_bottom(img, "陈治如睡觉打卡成功", font_path, font_size=280, align="center", margin=15)
else:
    img = put_chinese_text_bottom(img, "陈治如睡觉打卡失败", font_path, font_size=280, align="center", margin=15)


cv2.namedWindow('demo', 0)
cv2.imshow("demo", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [12]:
pip install dlib

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--
     ---------------------------------------- 0.0/3.3 MB ? eta -:--:--

  error: subprocess-exited-with-error
  
  × Building wheel for dlib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [41 lines of output]
      running bdist_wheel
      running build
      running build_ext
      
      
                         CMake is not installed on your system!
      
          Or it is possible some broken copy of cmake is installed on your system.
          It is unfortunately very common for python package managers to include
          broken copies of cmake.  So if the error above this refers to some file
          path to a cmake file inside a python or anaconda or miniconda path then you
          should delete that broken copy of cmake from your computer.
      
          Instead, please get an official copy of cmake from one of these known good
          sources of an official cmake:
              - cmake.org (this is how windows users should get cmake)
              - apt install cmake (for Ubuntu or Debian based systems)
             